## Ejemplo dos: Ajuste fino de un modelo LLM con OpenChat 3.5 y Bitext-travel-llm-chatbot-training-dataset

Utalizaremos a OpenChat 3.5 y la base de datos de Bitext-travel-llm-chatbot-training-dataset, para hacer una prueba inicial de ajuste fino y revisar las alucinaciones del modelo cuando se le pregunta por un tema que no conoce o se le preugnta por una solucion bajo la cual no tiene dominio.


In [23]:
model_id = "openchat/openchat-3.5-0106"

from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_4bit=True,
    device_map="auto"
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 3/3 [00:13<00:00,  4.50s/it]
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [24]:
import pandas as pd
from datasets import load_dataset
dataset_1 = load_dataset("bitext/Bitext-travel-llm-chatbot-training-dataset")
df_1 = pd.DataFrame(dataset_1["train"])

In [36]:
df_1

,instruction,intent,category,tags,response
0,I want to know about my checked carry-on bagga...,check_baggage_allowance,BAGGAGE,BCIL,"To find out your checked baggage allowance, pl..."
1,I'm looking for information about my checked c...,check_baggage_allowance,BAGGAGE,BCLQZ,To check the details of your checked baggage a...
2,I'm looking for information about my fucking c...,check_baggage_allowance,BAGGAGE,BCILW,"To find out your checked baggage allowance, pl..."
3,I have to see the fuking checked carry-on bagg...,check_baggage_allowance,BAGGAGE,BCILPWZ,"To review your checked baggage allowance, plea..."
4,need to see the fucking checkedcarry-on baggag...,check_baggage_allowance,BAGGAGE,BCILQWZ,"To check your baggage allowance, please follow..."
...,...,...,...,...,...
31653,wanna find a travel insurance could ya help me,search_trip_insurance,TRIP,BCILPQ,To obtain the most suitable travel insurance f...
31654,"I'd like to find a fucking trip insurance, whe...",search_trip_insurance,TRIP,BCILPWZ,To locate the optimal travel insurance that me...
31655,"I have to find fucking tfavel insurances, how ...",search_trip_insurance,TRIP,BCILMWZ,To locate the optimal travel insurance that me...
31656,I need to search for fucking travel insurances...,search_trip_insurance,TRIP,BCILMW,"To secure suitable travel insurance, please fo..."


In [26]:
# Format without metadata
def format_for_openchat(example):
    return {
        "text": f"<|system|>\nYou are a travel expert assistant.\n<|user|>\n{example['instruction']}\n<|assistant|>\n{example['response']}"
    }

In [27]:
# Format including metadata (intent and category)
def format_with_metadata(example):
    return {
        "text": (
            f"<|system|>\nYou are a travel expert assistant. User intent: {example['intent']}. Category: {example['category']}.\n"
            f"<|user|>\n{example['instruction']}\n"
            f"<|assistant|>\n{example['response']}"
        )
    }
dataset = dataset_1.map(format_with_metadata)


Map: 100%|██████████| 31658/31658 [00:05<00:00, 5372.78 examples/s]


In [28]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'intent', 'category', 'tags', 'response', 'text'],
        num_rows: 31658
    })
})

In [29]:
# Paso 5: Preparar el modelo para entrenamiento con LoRA en 4-bit
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training

# Permite entrenar solo capas seleccionadas en modelos 4-bit
model = prepare_model_for_kbit_training(model)

In [30]:
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=32,
    lora_alpha=64,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)


In [31]:
# Paso 6: Entrenamiento supervisado con TRL y SFTTrainer
from transformers import TrainingArguments
from trl import SFTTrainer

model = get_peft_model(model, lora_config)

training_args = TrainingArguments(
    output_dir="./openchat-finetuned-bitext",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    fp16=True,
    report_to="none"
)

# Usamos la columna "text" ya preformateada para entrenamiento
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    args=training_args
)


Truncating train dataset: 100%|██████████| 31658/31658 [00:00<00:00, 77539.63 examples/s]
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [32]:
# Inicia el entrenamiento
trainer.train()

/mnt/sda1/prueb/LLM_study/.venv/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/mnt/sda1/prueb/LLM_study/.venv/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/mnt/sda1/pr

Step,Training Loss
10,0.940800
20,0.534900
30,0.454900
40,0.442600
50,0.399200
60,0.390200
70,0.372200
80,0.359100
90,0.353300
100,0.349700


/mnt/sda1/prueb/LLM_study/.venv/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/mnt/sda1/prueb/LLM_study/.venv/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=11874, training_loss=0.17390654825774712, metrics={'train_runtime': 61650.5888, 'train_samples_per_second': 1.541, 'train_steps_per_second': 0.193, 'total_flos': 8.681052038665175e+17, 'train_loss': 0.17390654825774712})

In [33]:
# Paso 7: Guardar el modelo y el tokenizer
trainer.save_model("./openchat-finetuned-bitext")
tokenizer.save_pretrained("./openchat-finetuned-bitext")

('./openchat-finetuned-bitext/tokenizer_config.json',
 './openchat-finetuned-bitext/special_tokens_map.json',
 './openchat-finetuned-bitext/chat_template.jinja',
 './openchat-finetuned-bitext/tokenizer.json')

In [42]:
#Paso 8 (opcional): probar el modelo entrenado
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

prompt = (
    "<|system|>\nYou are a travel expert assistant\n"
    "<|user|>\nI'm looking for a trip insurance help me to find it\n"
    "<|assistant|>\n"
)

print(pipe(prompt, max_new_tokens=200)[0]['generated_text'])


Device set to use cuda:0
/mnt/sda1/prueb/LLM_study/.venv/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/mnt/sda1/prueb/LLM_study/.venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<|system|>
You are a travel expert assistant
<|user|>
I'm looking for a trip insurance help me to find it
<|assistant|>
To secure the most suitable travel insurance for your requirements, please adhere to these directions:

1. Navigate to {{WEBSITE_URL}}.
2. Fill in the relevant trip information such as {{ORIGIN_CITY}}, {{DESTINATION_CITY}}, and the dates of travel.
3. Proceed to the {{SEARCH_TRIP_INSURANCE_SECTION}} area to view the available insurance options.
4. Examine the details of each coverage plan and select the insurance policy that meets your requirements.
5. Finalize the transaction by following the subsequent instructions.

Should you require any more detailed assistance, do not hesitate to inquire further.
